# Day 17: Set and Forget

In [1]:
import numpy as np
from intcomputer import Intcomputer

In [2]:
with open('input.txt') as f:
    code = [int(i) for i in f.readline().split(',')]

## Part I

In [3]:
def get_camera_image():
    cpu = Intcomputer(code)

    img = ''
    state = 'INIT'

    while state != 'HALT':
        state, output = cpu.execute()
        if output: img += chr(output)
            
    return img.rstrip('\n')

In [4]:
img = get_camera_image()

In [5]:
print(img)

........................#############......................
........................#...........#......................
........................#...........#......................
........................#...........#......................
........................#...........#############..........
........................#.......................#..........
........................#.......................#..........
........................#.......................#..........
........................#.......................#..........
........................#.......................#..........
........................#.......................#..........
........................#.......................#..........
....................#####.......................#..........
....................#...........................#..........
....................#.........................#####.#......
....................#.........................#.#.#.#......
........#####.....#########.............

In [6]:
def is_crossing(i, j, matrix):
    try:
        return (matrix[i, j] != '.'
                and matrix[i + 1, j] != '.'
                and matrix[i - 1, j] != '.'
                and matrix[i, j + 1] != '.'
                and matrix[i, j - 1] != '.')
    except IndexError:
        return False

In [7]:
matrix = np.array([[tile for tile in line] for line in img.split('\n')])

In [8]:
crossings = [(i, j)
             for i, row in enumerate(matrix)
             for j, tile in enumerate(row) if is_crossing(i, j, matrix)]

In [9]:
# sum of coordinate-products of crossing points
sum([i * j for i, j in crossings])

6672

## Part II

In [10]:
U = (-1, 0)
L = (0, -1)
D = (1, 0)
R = (0, 1)

In [11]:
def get_starting_parameters(matrix):
    i, j = np.where(matrix == '^')
    return (i[0], j[0]), U

In [12]:
def count_tiles(position, direction):
    count = 0
    current = position
    
    while True:
        try:
            current = tuple(np.add(current, direction))
            if matrix[current] in '#':
                count += 1
            else:
                break
        except IndexError:
            break
    
    return count

In [13]:
def turn_right(orientation):
    return {U: R, R: D, D: L, L: U}[orientation]

In [14]:
def turn_left(orientation):
    return {U: L, L: D, D: R, R: U}[orientation]

In [15]:
def move(position, orientation, steps):
    for _ in range(steps):
        position = tuple(np.add(position, orientation))
    return position

In [16]:
def find_path(matrix):
    position, orientation = get_starting_parameters(matrix)
    steps = []

    while True:
        tiles = count_tiles(position, orientation)

        if tiles > 0:
            steps.append(tiles)
            position = move(position, orientation, tiles)
        else:
            # look right
            if count_tiles(position, turn_right(orientation)):
                steps.append('R')
                orientation = turn_right(orientation)
            # look left
            elif count_tiles(position, turn_left(orientation)):
                steps.append('L')
                orientation = turn_left(orientation)
            # endpoint reached
            else:
                break
                
    return steps

In [17]:
steps = find_path(matrix)

In [18]:
steps[:10]

['L', 12, 'R', 4, 'R', 4, 'R', 12, 'R', 4]

In [19]:
def split(sequence, maxlen=12):
    """Return three subsequences - building blocks of the sequence."""
    is_match = lambda i, j, l: l > 0 and sequence[i:i + l] == sequence[j:j + l]

    a_pos = b_pos = c_pos = 0
    a_len = b_len = c_len = 0
    i = 0

    while i < len(sequence):
        if is_match(i, a_pos, a_len):
            i += a_len
        elif  is_match(i, b_pos, b_len):
            i += b_len
        elif is_match(i, c_pos, c_len):
            i += c_len
        elif c_len < maxlen:
            if c_len == 0: c_pos = i
            c_len += 1
            i = 0
        elif b_len < maxlen:
            if b_len == 0: b_pos = i
            b_len += 1
            c_len = 0
            i = 0
        elif a_len < maxlen:
            a_len += 1
            b_len = c_len = 0
            i = 0
        else:
            return None

    return (sequence[a_pos:a_pos + a_len],
            sequence[b_pos:b_pos + b_len],
            sequence[c_pos:c_pos + c_len])


In [20]:
a, b, c = split(steps)
a, b, c

(['L', 12, 'R', 4, 'R', 4],
 ['R', 12, 'R', 4, 'L', 6, 'L', 8, 'L', 8],
 ['R', 12, 'R', 4, 'L', 12])

In [21]:
main = []
i = 0

while i < len(steps):
    if steps[i:i + len(a)] == a:
        main.append('A')
        i += len(a)
    elif steps[i:i + len(b)] == b:
        main.append('B')
        i += len(b)
    elif steps[i:i + len(c)] == c:
        main.append('C')
        i += len(c)
    else:
        raise ValueError

In [22]:
main

['A', 'C', 'C', 'B', 'B', 'A', 'A', 'C', 'C', 'B']

In [23]:
def programs():
    for seq in [main, a, b, c]:
        yield [ord(n) for n in ','.join(map(str, seq)) + '\n']
        
    yield [ord('n'), ord('\n')]

In [24]:
progs = programs()

cpu = Intcomputer(code)
cpu.memory[0] = 2

output = [[]]
state = 'INIT'

while state != 'HALT':
    state, out = cpu.execute()
    
    if state == 'OUTPUT':
        output[-1].append(out)
    
    if state == 'WAITING':
        prog = next(progs)
        
        while state == 'WAITING':
            state, out = cpu.execute(prog.pop(0))
        
        if state == 'OUTPUT':
            output.append([out])

In [25]:
# last number the program outputs
output[-1][-1]

923017